In [ ]:
from pynq import Overlay, allocate
import numpy as np
import time

# Load the overlay (replace with the correct path if needed)
ol = Overlay("sistema.bit")

In [ ]:
# Define the array size: 2^13 = 8192 elements
size = 8192

# Allocate contiguous memory for input and output buffers
input_buffer = allocate(shape=(size,), dtype=np.float32)
output_buffer = allocate(shape=(size,), dtype=np.float32)

# Initialize input data (for example, with random numbers)
input_buffer[:] = np.random.rand(size).astype(np.float32)


gain = np.float32(2.0)  # Set your gain value

In [ ]:
# Obtener la instancia del IP (asegúrate de que el nombre coincida con el del diseño)
kr = ol.array_gain_0

# Ver el mapa de registros (opcional)
print("Mapa de Registros:")
print(kr.register_map)

In [ ]:
# Escribir las direcciones físicas y el parámetro a los registros utilizando el mapa:
kr.write(kr.register_map.input_r_1.address, input_buffer.physical_address)
#kr.write(kr.register_map.input_r_2.address, 0)  # Suponemos que la dirección cabe en 32 bits

kr.write(kr.register_map.output_r_1.address, output_buffer.physical_address)
#kr.write(kr.register_map.output_r_2.address, 0)

# Convert the float 'gain' into its 32-bit integer representation and cast to int:
gain_int = int(np.frombuffer(gain.tobytes(), dtype=np.uint32)[0])
kr.write(kr.register_map.gain.address, gain_int)

# Start the kernel by writing '1' to the control register at offset 0x00
kr.register_map.CTRL.AP_START = 1
time.sleep(1)



In [ ]:
# Now, output_buffer should contain the results (each element multiplied by 'gain')
print("First 10 input values  :", input_buffer[:10])
print("First 10 output values :", output_buffer[:10])
print("Gain used              :", gain)

In [ ]:
# When finished, free the allocated buffers (if desired)
input_buffer.freebuffer()
output_buffer.freebuffer()